In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import statistics as stat

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/student-performance-data/student_data.csv')
df = df.dropna()
df.columns

In [ ]:
y1 = df['G1']
y2 = df['G2']
y3 = df['G3']
ytot = y1 + y2 + y3
y = ytot / 58

In [ ]:
sns.scatterplot(x = df['absences'], y = y)

In [ ]:
X = df.drop(columns = ['G1', 'G2', 'G3'])
X.columns
X.head()
XH = X

In [ ]:
"""ALL CATS: ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']"""
cats = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
#XNoCats = X.drop(columns = cats)
#XNoCats.head()

In [ ]:
#ONEHOTENC TEST
transformer = make_column_transformer(
    (OneHotEncoder(), cats),
    remainder='passthrough')

transformed = transformer.fit_transform(df)
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
XH = transformed_df
XH.head()

In [ ]:
%%time

N = tf.keras.layers.Normalization()
N.adapt(np.array(XH))
linear_model = tf.keras.Sequential([
    N,
    layers.Dense(units=5, activation = 'softmax')
])
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00002),
    loss='mean_absolute_error')

In [ ]:
%%time
history = linear_model.fit(
    XH,
    y,
    epochs=100,
    # Suppress logging.
    verbose=2,
    # Calculate validation results on 10% of the training data.
    validation_split = 0.1)

In [ ]:
linear_model.evaluate(
    XH, y, verbose=1)

In [ ]:
linear_model.save('MDL4')
MDL4 = tf.keras.models.load_model('MDL4')

In [ ]:
linear_model.save('MDL3')
MDL = tf.keras.models.load_model('MDL3')

In [ ]:
MDL.evaluate(
    XH, y, batch_size=None, verbose=0)

In [ ]:
#If it doesn't work well --> high bias. Check last entries for high variance
predModel =  MDL4.predict(X)
print("FIRST 20")
for i in range(20):
    pred = round(predModel[i][0] * 58)
    yi = round(y[i] * 58)
    dif = round(abs(yi-pred))
    print(f"Actual: {yi}, Pred: {pred}, Diff: {dif}")
print("LAST 20")
for i in range(len(X)-20, len(X)):
    pred = round(predModel[i][0] * 58)
    yi = round(y[i] * 58)
    dif = round(abs(yi-pred))
    print(f"Actual: {yi}, Pred: {pred}, Diff: {dif}")